# Modèles non paramétriques et cross validation

### Auteur : [Menyssa CHERIFA](https://www.linkedin.com/in/menyssacherifa/)
### Projet : [Mimic](https://github.com/mcherifa/Mimic) 




### 1 - Importation des librairies nécessaires

In [3]:
# Path scripts
dest_r <- "/home/mcherifa/Mimic/scripts/R/toolbox/"

# Path data 
dest_data <- "/home/mcherifa/Mimic/data/clean/"

source(paste0(dest_r,"packages.R"))
source(paste0(dest_r,"fonctions.R"))
source(paste0(dest_r,"models.R"))
source(paste0(dest_r,"cross_validation.R"))

# Chargement des données 
df_wide <- readRDS(paste0(dest_data,"fichier_wide_periode.rds"))


# Management
df_wide <- df_wide[which(df_wide$eevent == 0),]
df_wide <- subset(df_wide, select = c( -id ,-periode, - identif,-eevent))

# Variables factors
df_wide[,c(1:5)]  <- data.frame(lapply(df_wide[,c(1:5)], as.factor))



Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

------------------------------------------------------------------------------
data.table + dplyr code now lives in dtplyr.
Please library(dtplyr)!
------------------------------------------------------------------------------

Attaching package: ‘data.table’

The following objects are masked from ‘package:xts’:

    first, last

The following objects are masked from ‘package:dplyr’:

    between, first, last

The follo

### 2 - Preproscessing des données

les covariables sont résumées par la moyenne et la variance. 


In [4]:

df_wide_numeric_resume = temp = NULL

# HR 

df_wide_numeric <- t(df_wide[,10:69])
df_wide_numeric_resume <- data.frame(m_hr = apply(df_wide_numeric,2,mean),
                                v_hr = apply(df_wide_numeric,2,var))
# SPO2

df_wide_numeric <- t(df_wide[,70:129])
temp <- data.frame(m_sp = apply(df_wide_numeric,2,mean),
                   v_sp = apply(df_wide_numeric,2,var))
df_wide_numeric_resume <- cbind(df_wide_numeric_resume,temp)

# abpsys

df_wide_numeric <- t(df_wide[,130:189])
temp <- data.frame(m_sys = apply(df_wide_numeric,2,mean),
                   v_sys = apply(df_wide_numeric,2,var))
df_wide_numeric_resume <- cbind(df_wide_numeric_resume,temp)

# abpdias

df_wide_numeric <- t(df_wide[,190:249])
temp <- data.frame(m_dias = apply(df_wide_numeric,2,mean),
                   v_dias = apply(df_wide_numeric,2,var))
df_wide_numeric_resume <- cbind(df_wide_numeric_resume,temp)

# abpmean

df_wide_numeric <- t(df_wide[,250:309])
temp <- data.frame(m_mean = apply(df_wide_numeric,2,mean),
                   v_mean = apply(df_wide_numeric,2,var))
df_wide_numeric_resume <- cbind(df_wide_numeric_resume,temp)

# resp

df_wide_numeric <- t(df_wide[,310:369])
temp <- data.frame(m_resp = apply(df_wide_numeric,2,mean),
                   v_resp = apply(df_wide_numeric,2,var))
df_wide_numeric_resume <- cbind(df_wide_numeric_resume,temp)

# ratio

df_wide_numeric <- t(df_wide[,370:429])
temp <- data.frame(m_ratio= apply(df_wide_numeric,2,mean),
                   v_ratio = apply(df_wide_numeric,2,var))

df_wide_numeric_resume <- cbind(df_wide_numeric_resume,temp)
df_wide_resume <- cbind(df_wide[,1:9] , df_wide_numeric_resume)

head(df_wide_resume)

,gender,event,ventilation,amine,admission_type_descr,age,bmi,sapsi_first,sofa_first,m_hr,⋯,m_sys,v_sys,m_dias,v_dias,m_mean,v_mean,m_resp,v_resp,m_ratio,v_ratio
2,F,0,0,1,EMERGENCY,87,34.85214,15,8,93.43833,⋯,120.4350,33.21350,41.02667,5.013853,65.72833,11.73325,23.86167,58.865116,0.6678118,0.0081411268
3,F,0,0,1,EMERGENCY,87,34.85214,15,8,93.55833,⋯,122.3467,606.61236,42.92833,87.539353,70.39333,60.12368,26.39167,6.529590,0.7523539,0.0063449359
4,F,0,0,1,EMERGENCY,87,34.85214,15,8,105.13833,⋯,118.9400,82.12007,45.46833,11.119150,71.67000,25.71739,28.30167,9.369658,0.6836477,0.0038044747
10,F,0,0,1,EMERGENCY,76,26.48931,15,9,110.52333,⋯,126.1367,54.78338,62.51167,7.561048,84.73500,17.89113,28.92167,5.014946,0.7664255,0.0009844666
11,F,0,0,1,EMERGENCY,76,26.48931,15,9,113.86167,⋯,101.4533,272.70185,51.46500,58.736551,70.42833,70.86071,30.68667,4.730328,0.6186069,0.0053789897
12,F,0,0,1,EMERGENCY,76,26.48931,15,9,107.28833,⋯,112.5983,110.58898,55.26167,19.203760,74.66333,46.85084,28.28833,5.930201,0.6951250,0.0027436730


### 3 - Monte Carlo cross validation 
            ( Repeated random sub-sampling validation )
            
Cette méthode consiste à générer de manière aléatoire un echantillon d'apprentissage et de validation. A chaque génération, un modèle est ajusté   sur les données d'apprentissage et la précision est calculée sur l'échantillon de validation. 

L'avantage de cette méthode est que les sous échantillons ne sont pas fixes donc plus le nombre de simulation est grand pus on s'approche de la méthode  __*leave-one-out cross-validation*__ mais l'inconvéniant est que certaines observations peuvent ne pas servir à la validation du modèle tandis que d'autres peuvent être sélectionnées plusieurs fois. 

In [ ]:
RES <- NULL

for ( i in 1:1000){

    # Bootstrap
    
    train_test <- bootstrapping(df = df_wide_resume )
    
    # Modele
    
    RES <- rbind(RES,deeplearning(liste = train_test))

}
RES$iter <- 1:1000
temp <- data.frame(matrix(unlist(RES), ncol = 6, nrow = 1000))

colnames( temp ) <- c( "AUC",  "CI_lower",  "CI_upper",  "ACCURACY",
                        "Time_to_predict", "iter" )

quantile(temp$AUC)

write.table(temp, file = "/home/mcherifa/Mimic/resultats/monte_carlo.csv", 
            sep = "\t", row.names = F)


# Graphique AUC 

pd <- position_dodge(0.1) 
temp$coloration <- ifelse(temp$AUC==1,"1","0")

ggplot(temp, aes(x=iter, y=AUC, colour = coloration)) + theme_bw()+ 
    geom_errorbar(aes(ymin=CI_lower, ymax=CI_upper), width=.1, position=pd) +
    geom_point(position=pd)
ggsave("/home/mcherifa/Mimic/resultats/monte_carlo_auc.png",
 plot = last_plot())


### 4 - k-fold cross-validation
            
Cette méthode consiste à diviser l'échantillon original en k échantillons, puis on sélectionne un des k échantillons comme ensemble de validation et les (k-1) autres échantillons constitueront l'ensemble d'apprentissage. On calcule comme dans la première méthode le score de performance. Puis on répète l'opération en sélectionnant un autre échantillon de validation parmi les (k-1) échantillons qui n'ont pas encore été utilisés pour la validation du modèle. L'opération se répète ainsi k fois pour qu'en fin de compte chaque sous-échantillon ait été utilisé exactement une fois comme ensemble de validation. La moyenne des k erreurs quadratiques moyennes est enfin calculée pour estimer l'erreur de prédiction.

In [5]:
RES_fold <-  NULL  
RES_fold <- k_fold_meny(df = df_wide_resume, k = 10)

  |                                                                      |   0%

Warning message in ci.auc.roc(roc, ...):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”Warning message in ci.auc.roc(AUC):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”

  |==============                                                        |  20%

Warning message in ci.auc.roc(roc, ...):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”Warning message in ci.auc.roc(AUC):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”

  |===================================                                   |  50%

Warning message in ci.auc.roc(roc, ...):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”Warning message in ci.auc.roc(AUC):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”

  |==========================================                            |  60%

Warning message in ci.auc.roc(roc, ...):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”Warning message in ci.auc.roc(AUC):
“ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading.”

  |======================================================================| 100%

In [6]:
RES_fold

AUC,CI_lower,CI_upper,ACCURACY,Time_to_predict
1,1,1,1,7.580014
0.9666667,0.9013345,1,0.9983278,4.959797
1,1,1,1,5.221931
0.5,0.5,0.5,0.9795571,5.310668
0.78125,0.6557272,0.9067728,0.9878893,5.024281
1,1,1,1,5.260179
1,1,1,1,4.577968
0.5,0.5,0.5,0.9716667,5.229117
0.9285714,0.8334619,1,0.9964789,5.422684
0.5333333,0.4680012,0.5986655,0.9754386,5.59531
